# Лабораторная работа № 4

## Постановка задачи

### Часть 1 
- Реализовать алгоритм базовой модели Идена; 
- Визуализировать рост кластера; 
- В лог-лог масштабе построить зависимости $N(R_{гир})$ и $N_s(R_{гир})$; 
- Определить размерность $d$ и $d_s$.

### Часть 2
- Реализовать алгоритм модели экранированного роста;
- Определить размерность кластера $d$ для случаев: $\psi < 2$, $\psi = 2$ и $\psi > 2$.

## Часть 1

### Реализация алгоритма базовой модели Идена

#### Описание

Рост кластера имитируется последовательным заполнением узлов квадратной решетки по периметру кластера.

В начальный момент времени кластер состоит всего из одной частицы ("зерна" роста). В дальнейшим рост кластера происходит за счет последовательного присоединения новых частиц в узлах по его периметру.

#### Структура кластеров Идена

Зависимость количества частиц в кластере и его радиусом гирации $R_g$ описывается соотношением $N \sim R_g^d$, где величина $d$ практически совпадает с евклидовой размерностью.

Радиус гирации определяется, как $R_g = \dfrac{1}{N}\sqrt{\sum\limits_{i=1}^{N} (\vec{r}_i - \vec{r})^2}$, где $\vec{r} = \dfrac{1}{N}\sum\limits_{i=1}^{N} \vec{r}_i$ соответствует координатам центра масс кластера.

Поверхность кластеров Идена имеет фрактальные свойства, а число узлов на периметре растет как $N_s \sim R_g^{d_f}$, где $d_f > 1$ $-$ фрактальная размерность периметра

Алгоритм базовой модели Идена был реализован в отдельном модуле [IdenCluster.py](https://github.com/AlexeyMakurin/Computer-technologies-in-physics/blob/main/IdenCluster.py). Здесь представлены результаты его работы.

In [1]:
from IdenCluster import IdenCluster

Создание сетки размером $n \times n$ с "зерном" роста в центре.

In [8]:
n=21
cluster = IdenCluster(n)

Запуск роста кластера.

Параметры
- records=1: запись стадий роста кластера для создания анимации;
- frame=10: сколько новых частиц будет содержать фрейм.

In [9]:
cluster.growth(records=1, frame=10)

Построение анимации.

In [11]:
cluster.vizualization()

Создадим новый кластер, и визуализируем его, но без демонстрации роста. В этом случае параметр records не указывается.

In [13]:
new_cluster = IdenCluster(101)
new_cluster.growth()
new_cluster.vizualization()

Цвет кластера меняется синусоидально, параметром, определяющим значение, является растояния от "зерна роста" до текущей частицы.

In [ ]:
#![image](images/iden_cluster_151.svg)